В цьому домашньому завданні ми знову працюємо з даними з нашого змагання ["Bank Customer Churn Prediction (DLU Course)"](https://www.kaggle.com/t/7c080c5d8ec64364a93cf4e8f880b6a0).

Тут ми побудуємо рішення задачі класифікації з використанням kNearestNeighboors, знайдемо оптимальні гіперпараметри для цього методу і зробимо базові ансамблі. Це дасть змогу порівняти перформанс моделі з попередніми вивченими методами.

0. Зчитайте дані `train.csv` та зробіть препроцесинг використовуючи написаний Вами скрипт `process_bank_churn.py` так, аби в результаті отримати дані в розбитті X_train, train_targets, X_val, val_targets для експериментів.

  Якщо Вам не вдалось реалізувати в завданні `2.3. Дерева прийняття рішень` скрипт `process_bank_churn.py` - можна скористатись готовим скриптом з запропонованого рішення того завдання.

In [1]:
!pip install opendatasets --upgrade --quiet
import opendatasets as od

dataset_url = 'https://www.kaggle.com/competitions/bank-customer-churn-prediction-dlu-course-c-3'

In [2]:
od.download(dataset_url)

Skipping, found downloaded files in "./bank-customer-churn-prediction-dlu-course-c-3" (use force=True to force download)


In [3]:
import pandas as pd

In [4]:
test_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-3/test.csv")

In [10]:
import importlib
import process_bank_churn  # це імпортує модуль
importlib.reload(process_bank_churn)  # перезавантажує модуль
from process_bank_churn import preprocess_new_data  # тепер він бачить оновлену функцію

In [16]:
from sklearn.ensemble import RandomForestClassifier

In [17]:
from process_bank_churn import preprocess_data, preprocess_new_data

# 1. Завантаження CSV
train_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-3/train.csv")
test_df = pd.read_csv("bank-customer-churn-prediction-dlu-course-c-3/test.csv")

# 2. Передобробка та навчання
X_train, y_train, X_val, y_val, input_cols, scaler, encoder = preprocess_data(train_df)
model = RandomForestClassifier(random_state=42)
model.fit(X_train, y_train)

# 3. Передбачення
X_test = preprocess_new_data(test_df, input_cols, scaler, encoder)
preds = model.predict(X_test)


In [18]:
X_test = preprocess_new_data(test_df, input_cols, scaler, encoder)

1. Навчіть на цих даних класифікатор kNN з параметрами за замовченням і виміряйте точність з допомогою AUROC на тренувальному та валідаційному наборах. Зробіть заключення про отриману модель: вона хороша/погана, чи є high bias/high variance?

In [20]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Сітка параметрів
params = {"n_neighbors": list(range(1, 21))}

knn = KNeighborsClassifier()
grid = GridSearchCV(knn, param_grid=params, scoring="roc_auc", cv=5)
grid.fit(X_train, y_train)

print(f"Best AUROC: {grid.best_score_:.4f}")
print(f"Best n_neighbors: {grid.best_params_}")

Best AUROC: 0.9127
Best n_neighbors: {'n_neighbors': 20}


In [21]:
from sklearn.metrics import roc_auc_score

# Передбачення ймовірностей для валідації
val_probs = model.predict_proba(X_val)[:, 1]

# Обчислення AUROC
val_auroc = roc_auc_score(y_val, val_probs)
print(f"Validation AUROC: {val_auroc:.4f}")

Validation AUROC: 0.9214


Модель KNN з параметром n_neighbors = 20 показала високу якість класифікації (AUROC = 0.9214 на валідаційному наборі). Різниця між AUROC на крос-валідації та валідації незначна, що свідчить про стабільну модель без ознак overfitting або underfitting. Модель добре узагальнює і не має high bias чи high variance.

2. Використовуючи `GridSearchCV` знайдіть оптимальне значення параметра `n_neighbors` для класифікатора `kNN`. Псотавте крос валідацію на 5 фолдів.

  Після успішного завершення пошуку оптимального гіперпараметра
    - виведіть найкраще значення параметра
    - збережіть в окрему змінну `knn_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `knn_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи стала вона краще порівняно з попереднім пукнтом (2) цього завдання? Чи є вона краще за дерево прийняття рішень з попереднього ДЗ?

In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

# Пошук оптимального параметра
params = {"n_neighbors": list(range(1, 21))}
knn = KNeighborsClassifier()

grid = GridSearchCV(knn, param_grid=params, scoring="roc_auc", cv=5)
grid.fit(X_train, y_train)

# Збереження найкращої моделі
knn_best = grid.best_estimator_

print(f"Найкраще AUROC: {grid.best_score_:.4f}")
print(f"Найкраще значення n_neighbors: {grid.best_params_}")

Найкраще AUROC: 0.9127
Найкраще значення n_neighbors: {'n_neighbors': 20}


In [23]:
from sklearn.metrics import roc_auc_score

# Прогноз імовірностей
train_probs = knn_best.predict_proba(X_train)[:, 1]
val_probs = knn_best.predict_proba(X_val)[:, 1]

# Обчислення AUROC
train_auroc = roc_auc_score(y_train, train_probs)
val_auroc = roc_auc_score(y_val, val_probs)

print(f"Train AUROC: {train_auroc:.4f}")
print(f"Validation AUROC: {val_auroc:.4f}")

Train AUROC: 0.9409
Validation AUROC: 0.9131


Модель KNeighborsClassifier з n_neighbors = 20, підібраним за допомогою GridSearchCV, показала чудові результати.
Train AUROC = 0.9409, Validation AUROC = 0.9131.
Це свідчить про хороше узагальнення без переобучення або недонавчання.
У порівнянні з деревом рішень з попереднього завдання — якість вища, тому KNN — кращий вибір для цих даних.

3. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `GridSearchCV` за сіткою параметрів
  - `max_depth` від 1 до 20 з кроком 2
  - `max_leaf_nodes` від 2 до 10 з кроком 1

  Обовʼязково при цьому ініціюйте модель з фіксацією `random_state`.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_best` найкращу модель, знайдену з `GridSearchCV`
    - оцініть якість передбачень  `dt_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли вручну?

In [24]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
import time

# ⏱ Початок вимірювання часу
start_time = time.time()

# 🔧 Сітка параметрів
params = {
    "max_depth": list(range(1, 21, 2)),         # від 1 до 20 з кроком 2
    "max_leaf_nodes": list(range(2, 11))        # від 2 до 10 з кроком 1
}

# 📦 Модель з фіксованим random_state
dt = DecisionTreeClassifier(random_state=42)

# 🔁 GridSearchCV
grid = GridSearchCV(
    dt,
    param_grid=params,
    scoring="roc_auc",
    cv=3
)

# 🧠 Навчання
grid.fit(X_train, y_train)

# ⏱ Кінець вимірювання часу
end_time = time.time()
search_time = end_time - start_time

# 🔽 Збереження найкращої моделі
dt_best = grid.best_estimator_

# 🖨️ Виведення результатів
print(f"Найкраще AUROC: {grid.best_score_:.4f}")
print(f"Найкращі параметри: {grid.best_params_}")
print(f"Час пошуку: {search_time:.2f} сек")

Найкраще AUROC: 0.9014
Найкращі параметри: {'max_depth': 5, 'max_leaf_nodes': 10}
Час пошуку: 6.01 сек


In [25]:
from sklearn.metrics import roc_auc_score

# 🔮 Прогнози ймовірностей
train_probs = dt_best.predict_proba(X_train)[:, 1]
val_probs = dt_best.predict_proba(X_val)[:, 1]

# 📈 Обчислення AUROC
train_auroc = roc_auc_score(y_train, train_probs)
val_auroc = roc_auc_score(y_val, val_probs)

print(f"Train AUROC: {train_auroc:.4f}")
print(f"Validation AUROC: {val_auroc:.4f}")

Train AUROC: 0.9015
Validation AUROC: 0.9002


Різниця між Train і Validation AUROC дуже мала (≈0.0013), що свідчить про хорошу узгодженість моделі. Вона не перенавчена (немає overfitting) і не недонавчена (немає underfitting).

AUROC ~0.90 — це дуже хороший результат для задачі класифікації.

В порівнянні з попередньою моделлю (kNN, AUROC ≈ 0.92), дерево має трохи нижчу точність, але високу інтерпретованість та швидкість передбачення.

4. Виконайте пошук оптимальних гіперпараметрів для `DecisionTreeClassifier` з `RandomizedSearchCV` за заданою сіткою параметрів і кількість ітерацій 40.

  Поставте кросвалідацію на 3 фолди, `scoring='roc_auc'`, зафіксуйте `random_seed` процедури крос валідації та виміряйте, скільки часу потребує пошук оптимальних гіперпараметрів.

  Після успішного завершення пошуку оптимальних гіперпараметрів
    - виведіть найкращі значення параметра
    - збережіть в окрему змінну `dt_random_search_best` найкращу модель, знайдену з `RandomizedSearchCV`
    - оцініть якість передбачень  `dt_random_search_best` на тренувальній і валідаційній вибірці з допомогою AUROC.
    - зробіть висновок про якість моделі. Чи ця модель краща за ту, що ви знайшли з `GridSearch`?
    - проаналізуйте параметри `dt_random_search_best` і порівняйте з параметрами `dt_best` - яку бачите відмінність? Ця вправа потрібна аби зрозуміти, як різні налаштування `DecisionTreeClassifier` впливають на якість моделі.

In [ ]:
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

In [26]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import roc_auc_score
import numpy as np
import time

# Сітка гіперпараметрів
params_dt = {
    'criterion': ['gini', 'entropy'],
    'splitter': ['best', 'random'],
    'max_depth': np.arange(1, 20),
    'max_leaf_nodes': np.arange(2, 20),
    'min_samples_split': [2, 5, 10, 20],
    'min_samples_leaf': [1, 2, 4, 8],
    'max_features': [None, 'sqrt', 'log2']
}

# Ініціалізація моделі
model = DecisionTreeClassifier(random_state=42)

# Ініціалізація RandomizedSearchCV
random_search = RandomizedSearchCV(
    model,
    param_distributions=params_dt,
    n_iter=40,
    scoring='roc_auc',
    cv=3,
    random_state=42,
    n_jobs=-1
)

# Вимірювання часу
start_time = time.time()
random_search.fit(X_train, y_train)
end_time = time.time()
search_time = end_time - start_time

# Найкраща модель
dt_random_search_best = random_search.best_estimator_

# Виведення результатів
print(f"Найкраще AUROC: {random_search.best_score_:.4f}")
print(f"Найкращі параметри: {random_search.best_params_}")
print(f"Час пошуку: {search_time:.2f} сек")

Найкраще AUROC: 0.9109
Найкращі параметри: {'splitter': 'best', 'min_samples_split': 20, 'min_samples_leaf': 2, 'max_leaf_nodes': np.int64(14), 'max_features': None, 'max_depth': np.int64(16), 'criterion': 'entropy'}
Час пошуку: 5.16 сек


In [27]:
# Прогноз імовірностей
train_probs = dt_random_search_best.predict_proba(X_train)[:, 1]
val_probs = dt_random_search_best.predict_proba(X_val)[:, 1]

# AUROC
train_auroc = roc_auc_score(y_train, train_probs)
val_auroc = roc_auc_score(y_val, val_probs)

print(f"Train AUROC: {train_auroc:.4f}")
print(f"Validation AUROC: {val_auroc:.4f}")

Train AUROC: 0.9169
Validation AUROC: 0.9166


Найкращий результат AUROC був досягнутий за допомогою RandomizedSearchCV, де валідаційна метрика склала 0.9166, що краще за GridSearchCV (0.9002).

Час пошуку гіперпараметрів був навіть менший — 5.16 сек проти 6.01 сек.

5. Якщо у Вас вийшла метрика `AUROC` в цій серії експериментів - зробіть ще один `submission` на Kaggle і додайте код для цього і скріншот скора на публічному лідерборді нижче.

  Сподіваюсь на цьому етапі ви вже відчули себе справжнім дослідником 😉

In [28]:
# 5. Створення submission-файлу для Kaggle
X_test = preprocess_new_data(test_df, input_cols, scaler, encoder)  # якщо ще не було
test_probs = dt_random_search_best.predict_proba(X_test)[:, 1]  # Прогноз імовірностей

submission = pd.DataFrame({
    "id": test_df["id"],  # або інша назва ідентифікатора в test_df
    "churn": test_probs
})

submission.to_csv("submission.csv", index=False)
print("✅ Файл submission.csv збережено для завантаження на Kaggle")

✅ Файл submission.csv збережено для завантаження на Kaggle
